In [1]:
import pandas as pd 

In [ ]:
#Número de escolas por cidade

In [ ]:
df = pd.read_csv(r'C:\Users\guilh\Documents\CENSO_ESCOLAR\2015\DADOS\ESCOLAS.csv',sep='|',encoding='Latin1',low_memory=False,
                usecols=['CO_ENTIDADE','CO_UF','CO_MUNICIPIO','TP_DEPENDENCIA','TP_SITUACAO_FUNCIONAMENTO'])

In [ ]:
#Selecionando as escolas com UF = 35 (Estado de SP), TP_DEPENDENCIA = 2 (Escolas Estaduais) e TP_SITUACAO_FUNCIONAMENTO (Escolas 'Em Atividade') 
df_sp = df[(df['CO_UF'] == 35)&(df['TP_DEPENDENCIA'] == 2)&(df['TP_SITUACAO_FUNCIONAMENTO'] == 1)]

In [ ]:
df_sp = df_sp.groupby('CO_MUNICIPIO').count().reset_index()[['CO_MUNICIPIO','CO_ENTIDADE']]

In [ ]:
#Número de estudantes

In [ ]:
chunk = pd.read_csv(r'C:\Users\guilh\Documents\CENSO_ESCOLAR\2015\DADOS\MATRICULA_SUDESTE.csv',sep='|',encoding='Latin1',low_memory=False,
                    usecols=['CO_PESSOA_FISICA','NU_IDADE','CO_UF','CO_MUNICIPIO','TP_DEPENDENCIA'],chunksize = 100000)

In [ ]:
#Selecionando as escolas com UF = 35 (Estado de SP) e TP_DEPENDENCIA = 2 (Escolas Estaduais)
ls = []
for data in chunk:
    data = data[(data['CO_UF'] == 35)&(data['TP_DEPENDENCIA'] == 2)]
    ls.append(data)
df_estud = pd.concat(ls)

In [ ]:
#Retirando duplicatas na coluna ID dos matriculados 
df_estud = df_estud.drop_duplicates(subset=['CO_PESSOA_FISICA'])

In [ ]:
#Agrupando por código de município e contando 
df_estud = df_estud.groupby(['CO_MUNICIPIO','NU_IDADE']).count().reset_index()[['CO_MUNICIPIO','CO_PESSOA_FISICA','NU_IDADE']]

In [ ]:
df_estud

In [ ]:
pd.merge(df_sp,df_estud,on=['CO_MUNICIPIO'])

In [ ]:
#Operacionalização do processo

In [2]:
writer = pd.ExcelWriter('info_censo_escolar.xlsx', engine='xlsxwriter')

In [3]:
for i in range(2015,2020):
    if (i==2015)|(i==2016)|(i==2017):
        df = pd.read_csv(rf'C:\Users\guilh\Documents\CENSO_ESCOLAR\{i}\DADOS\ESCOLAS.csv',sep='|',encoding='Latin1',low_memory=False,
                    usecols=['CO_ENTIDADE','CO_UF','CO_MUNICIPIO','TP_DEPENDENCIA','TP_SITUACAO_FUNCIONAMENTO'])
        df_sp = df[(df['CO_UF'] == 35)&(df['TP_DEPENDENCIA'] == 2)&(df['TP_SITUACAO_FUNCIONAMENTO'] == 1)]
        df_sp = df_sp.groupby('CO_MUNICIPIO').count().reset_index()[['CO_MUNICIPIO','CO_ENTIDADE']]
        chunk = pd.read_csv(rf'C:\Users\guilh\Documents\CENSO_ESCOLAR\{i}\DADOS\MATRICULA_SUDESTE.csv',sep='|',encoding='Latin1',low_memory=False,
                        usecols=['CO_PESSOA_FISICA','NU_IDADE','CO_UF','CO_MUNICIPIO','TP_DEPENDENCIA'],chunksize = 100000)
        ls = []
        for data in chunk:
            data = data[(data['CO_UF'] == 35)&(data['TP_DEPENDENCIA'] == 2)]
            ls.append(data)
        df_estud = pd.concat(ls)
        df_estud = df_estud.drop_duplicates(subset=['CO_PESSOA_FISICA'])
        df_estud = df_estud.groupby(['CO_MUNICIPIO','NU_IDADE']).count().reset_index()[['CO_MUNICIPIO','CO_PESSOA_FISICA','NU_IDADE']]
        df = pd.merge(df_sp,df_estud,on=['CO_MUNICIPIO'])
        print(i)
        df.to_excel(writer, index = False, sheet_name=f'Tabela_{i}')
    else:
        df = pd.read_csv(rf'C:\Users\guilh\Documents\CENSO_ESCOLAR\{i}\DADOS\ESCOLAS.csv',sep='|',encoding='Latin1',low_memory=False,
                    usecols=['CO_ENTIDADE','CO_UF','CO_MUNICIPIO','TP_DEPENDENCIA','TP_SITUACAO_FUNCIONAMENTO'])
        df_sp = df[(df['CO_UF'] == 35)&(df['TP_DEPENDENCIA'] == 2)&(df['TP_SITUACAO_FUNCIONAMENTO'] == 1)]
        df_sp = df_sp.groupby('CO_MUNICIPIO').count().reset_index()[['CO_MUNICIPIO','CO_ENTIDADE']]
        chunk = pd.read_csv(rf'C:\Users\guilh\Documents\CENSO_ESCOLAR\{i}\DADOS\MATRICULA_SUDESTE.csv',sep='|',encoding='Latin1',low_memory=False,
                        usecols=['ID_ALUNO','NU_IDADE','CO_UF','CO_MUNICIPIO','TP_DEPENDENCIA'],chunksize = 100000)
        ls = []
        for data in chunk:
            data = data[(data['CO_UF'] == 35)&(data['TP_DEPENDENCIA'] == 2)]
            ls.append(data)
        df_estud = pd.concat(ls)
        df_estud = df_estud.drop_duplicates(subset=['ID_ALUNO'])
        df_estud = df_estud.groupby(['CO_MUNICIPIO','NU_IDADE']).count().reset_index()[['CO_MUNICIPIO','ID_ALUNO','NU_IDADE']]
        df = pd.merge(df_sp,df_estud,on=['CO_MUNICIPIO'])
        print(i)
        df.to_excel(writer, index = False, sheet_name=f'Tabela_{i}')   

2015
2016
2017
2018
2019


In [4]:
writer.save()